In [1]:
import pandas as pd
import tokenize_uk
from typing import List
from ast import literal_eval

In [2]:
def read_stop_words(file):
    with open(file) as f:
        stop_words = f.read().split('\n')

    return stop_words

## Read dataset and create docs

In [3]:
df = pd.read_csv("data/reviews_preprocessed.csv")

In [4]:
df.head()

,Unnamed: 0,title,pos,neg,ratingValue,bestRating,rate
0,0,"['Ідеально', 'для', 'ділової', 'поїздки']","['Ідеально', 'для', 'ділової', 'поїздки', 'Гос...",['Nan'],10.0,10.0,1
1,1,"['Затишний', 'чистий', 'номер', 'з', 'усіма', ...","['Затишний', 'чистий', 'номер', 'з', 'усіма', ...","['При', 'бронюванні', 'вказала', 'час', 'прибу...",9.2,10.0,1
2,2,"['Все', 'сподобалося', 'Рекомендую']","['Чисто', 'тихо', 'комфортно', 'Зустрів', 'і',...","['На', 'барі', 'кава', 'тільки', '3', 'в', '1'...",9.6,10.0,1
3,3,"['Зручне', 'розташування', 'чудовий', 'вигляд'...","['Зручне', 'розташування', 'чудовий', 'вигляд'...",['Nan'],10.0,10.0,1
4,4,"['Все', 'чудово', '9,9', 'балів']","['Нові', 'апартаменти', 'на', 'останньому', 'п...","['Немає', 'терміналу', 'для', 'оплати', 'креди...",10.0,10.0,1


In [5]:
title = literal_eval(df.iloc[0]['title'])

In [6]:
' '.join(word for word in title)

'Ідеально для ділової поїздки'

In [7]:
def create_docs(data):
    docs = []
    for i in range(0, len(data)):
        text = ""
        
        title = literal_eval(data.iloc[i]['title'])
        if len(title) > 0 and title[0] != 'Nan':
            text += ' '.join(word.lower() for word in title)
            
        pos = literal_eval(data.iloc[i]['pos'])
        if len(pos) > 0 and pos[0] != 'Nan':
            text += ' '
            text += ' '.join(word.lower() for word in pos)
            
        neg = literal_eval(data.iloc[i]['neg'])
        if len(neg) > 0 and neg[0] != 'Nan':
            text += ' '
            text += ' '.join(word.lower() for word in neg)
            
        docs.append(text)
    return docs
        

In [8]:
docs = create_docs(df)

In [9]:
docs[:3]

['ідеально для ділової поїздки ідеально для ділової поїздки господар зустрів о шостій ранку привітний 22 поверх вид з вікна на море все поряд супермаркет розваги ресторани було пізнє виселення без доплати обов язково зупинюся тут наступного разу',
 'затишний чистий номер з усіма зручностями затишний чистий номер з усіма зручностями чудовий краєвид з вікна є можливість підігріти їжу та зробити чай каву при бронюванні вказала час прибуття о 7 ранку на що отримала відповідь за наявності про те що раніше 12 я заселитися не зможу дізналася тільки коли туди приїхала виявляється мені про це повідомили о 4.45 електронним листом коли я ще спала у потязі вважаю що слід попереджати хоча б за 12 годин щоб була можливість скоригувати плани також мені не дали хоча я і просила підтвердження оплати для оформлення відрядження',
 'все сподобалося рекомендую чисто тихо комфортно зустрів і провів приємний власник в номері холодильник в холі мікрохвильовка чайник все що на мій погляд необхідно на барі кава

### Create the IDF

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
stop_words = read_stop_words('ukrainian-stopwords.txt')

In [12]:
cv=CountVectorizer(max_df=0.9,stop_words=stop_words)
word_count_vector=cv.fit_transform(docs)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['дев', 'ласка', 'разу', 'ятий', 'ятнадцятий', 'ятнадцять', 'ять', 'ім'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [13]:
word_count_vector.shape

(11448, 28858)

Let's limit our vocabulary size to 10,000

In [14]:
cv=CountVectorizer(max_df=0.85,stop_words=stop_words,max_features=1000)
word_count_vector=cv.fit_transform(docs)
word_count_vector.shape

(11448, 1000)

### TfidfTransformer to Compute Inverse Document Frequency (IDF) 

In [15]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

Let's look at some of the IDF values:

In [16]:
tfidf_transformer.idf_

array([4.97501964, 4.34672111, 5.81305818, 6.05519823, 5.68221857,
       5.98894884, 4.97968165, 4.98907139, 6.53899518, 6.51701627,
       6.32030598, 5.2397122 , 6.39441395, 6.12614996, 6.94446029,
       6.09716243, 5.13617152, 6.73473976, 6.70807151, 6.81929714,
       7.16760384, 6.58445755, 6.79030961, 5.13617152, 5.91484087,
       5.42567674, 5.2397122 , 4.873387  , 5.76069019, 5.98894884,
       6.51701627, 6.1712704 , 6.41383204, 6.49551007, 5.83479816,
       6.68209602, 5.14716064, 5.81305818, 6.60798805, 6.2347838 ,
       5.90300641, 6.12614996, 6.12614996, 6.39441395, 5.96363103,
       4.89461922, 6.91167046, 6.84915011, 4.49345519, 6.15600293,
       5.67282883, 6.20252294, 5.98894884, 6.1712704 , 5.38983061,
       6.1712704 , 5.36205105, 6.65677821, 6.09716243, 6.21852328,
       6.70807151, 6.60798805, 4.78497604, 6.41383204, 4.24309907,
       5.25806133, 6.1712704 , 6.28521466, 6.45383737, 4.38723298,
       6.58445755, 5.96363103, 6.37536576, 6.08297779, 4.29121

### Computing TF-IDF and Extracting Keywords

In [17]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [18]:
# you only needs to do this once
feature_names=cv.get_feature_names()

# get the document that we want to extract keywords from
doc=docs[0]

#generate tf-idf for the given document
tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))

#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(tf_idf_vector.tocoo())

#extract only the top n; n here is 10
keywords=extract_topn_from_vector(feature_names,sorted_items,10)

# now print the results
print("\n===Review text===")
print(doc)
print("\n===Keywords===")
for k in keywords:
    print(k,keywords[k])


===Review text===
ідеально для ділової поїздки ідеально для ділової поїздки господар зустрів о шостій ранку привітний 22 поверх вид з вікна на море все поряд супермаркет розваги ресторани було пізнє виселення без доплати обов язково зупинюся тут наступного разу

===Keywords===
ідеально 0.513
господар 0.307
супермаркет 0.286
наступного 0.24
виселення 0.238
поверх 0.237
язково 0.229
обов 0.227
море 0.223
ранку 0.223


In [19]:
a = tf_idf_vector=tfidf_transformer.transform(cv.transform(docs[:1]))

In [20]:
cv.transform(docs[:2])

<2x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 37 stored elements in Compressed Sparse Row format>

In [21]:
sorted_items=sort_coo(a.tocoo())

In [22]:
sorted_items

[(989, 0.5133355602019788),
 (189, 0.30718850936240566),
 (822, 0.28639787793957844),
 (506, 0.23968334097176036),
 (104, 0.23810897393376995),
 (630, 0.23719056631955449),
 (972, 0.22917608962417788),
 (561, 0.2269577117146022),
 (474, 0.22349105254866322),
 (722, 0.22304869841803548),
 (720, 0.22304869841803548),
 (647, 0.2194572483161226),
 (100, 0.2063779389212204),
 (166, 0.1723737177754766),
 (675, 0.12548909253931087)]